In [28]:
import nbimporter
import math
import numpy as np
import sys
from collections import OrderedDict
from sortedcontainers import SortedDict
if('..' not in sys.path):
    sys.path.insert(0,'..')
from Maths.Action import Action
from Maths.State import State
from Maths.Cord import Cord
from Maze.Maze import Maze
from Agents.Prey import Prey
import random

class Queen(Prey):
    sid=0
    def __init__(self, maze):
        super().__init__("Queen",Queen.sid)  
        Queen.sid+=1
        self.S=State.ENTRANCE
        self.m=maze
        self.Pos=Cord(maze.getInitialX(),maze.getInitialY())
        d={}
        self.mapDict=OrderedDict()
        self.history={}
        self.history[-1]=self.Pos
        self.historyInView={}
        self.view=[]
        self.top,self.bottom,self.left,self.right=0,0,0,0
        self.oldQueen,self.oneQueen=self.Pos,self.Pos
        self.vulnerable=False
        #self.RandomWalk()
        
    def updateVulnerability(self,pList):
        north=Cord(self.Pos.X,self.Pos.Y+1)
        south=Cord(self.Pos.X,self.Pos.Y-1)
        east=Cord(self.Pos.X+1,self.Pos.Y)
        west=Cord(self.Pos.X-1,self.Pos.Y)
                
        if((north in pList) or (south in pList) or (east in pList) or (west in pList)):
            self.vulnerable=False
        else:
            self.vulnerable=True
        
    def NextMove(self):
        direction=self.RandomWalk()        
        self.Do(direction,self.m)
        
    def RandomWalk(self):
        now=self.getPos()
        possibleDirections=self.m.WhichWayIsClear(now)
        direction=possibleDirections[random.randint(0,len(possibleDirections)-1)]
        return direction
        
    def Do(self, direction, m):
        if(direction==0):
            self.MoveNorth(m)
        if(direction==1):
            self.MoveSouth(m)
        if(direction==2):
            self.MoveEast(m)
        if(direction==3):
            self.MoveWest(m)
        if(direction==4):
            self.StayStill(m)
        newC=self.getPos()
        if(newC in m.getExit()):
            self.S=State.EXIT
        else:
            self.S=State.CLEAR
            
    """def getSetView(self,maze, span):
        views=[]
        for i in range(0,maze.getHeight()):
            for j in range(0, maze.getWidth()):
                c= Cord(i,j)
                view=self.getView(c,span)
                if(view not in views):
                    views.append(view)
        return views"""
    
    def show_span(self,span):
        for row in span:
            print(row)
        print(" ")
    
    def combine(self,span1, t1):
        #print("In Combining at",t1) 
        span=self.view
        newDict=OrderedDict()
        if len(span)==0:
            span=span1
            #print("a")
            for a in range(len(span)):
                for b in range(len(span[a])):
                    self.mapDict[Cord(a,b)]=[span[a][b],0]
            #print("b")        
            for i in range(len(span)):
                for j in range(len(span[i])):
                    if span[i][j]==7:
                        queenPos=Cord(i,j)
            #print("c")
            for i in range(self.getTime()+1):
                self.historyInView[i]=queenPos
                
        #self.qList[0].history.get(p.savedTime)
        #print("queen's histry in map:") 
        #for key in self.historyInView:
        #    print(key,self.historyInView.get(key).CordToString())
        NewSpan=np.ones(((len(span1)+len(span)),(len(span1)+len(span))))
        #print("queen's view")
        #self.show_span(span)
        #print("ant's view")
        #self.show_span(span1)

        one_queen=[0,0]
        old_queen=[0,0]
        

        for i in range(len(span1)):
            for j in range(len(span1[i])):
                if span1[i][j]==7:
                    one_queen=[i,j]
                    
        #for i in range(len(span1)):
        #    for j in range(len(span1[i])):
        #        if span[i][j]==7:
        #            pos=Cord(i,j)
        
        pos=self.historyInView.get(t1)
        #print(pos,t1,len(self.historyInView))
        #print(pos.CordToString(), t1,len(self.historyInView))
        #if(pos==None):
        ##for t in self.historyInView:
        #    print("History in view",t,self.historyInView.get(t).CordToString())
        #print("Queen's pos at",t1," is",pos.CordToString(),"Actual:",self.history.get(t1).CordToString())
        """
        for a in range(len(span)):
            for b in range(len(span[a])):
                if Cord(a,b).equals(pos):
                    old_queen=[a,b] """
        old_queen=[pos.X,pos.Y]

        #print("In queen's view:",old_queen, "in ant's view:",one_queen, "at time ",t1)

        #print("Extensions",x_extension, y_extension)

        """print("x to the left:",old_queen[1])
        print("x to the right:",len(span[0])-old_queen[1]-1)
        print("y to the top:",old_queen[0])
        print("y to the bottom:",len(span)-old_queen[0]-1)    

        print("x to the left:",one_queen[1])
        print("x to the right:",len(span1[0])-one_queen[1]-1)
        print("y to the top:",one_queen[0])
        print("y to the bottom:",len(span1)-one_queen[0]-1)"""

        left,right,top,bottom=0,0,0,0
        
        self.top,self.bottom,self.left,self.right=top,bottom,left,right
        self.oldQueen,self.oneQueen=Cord(old_queen[0],old_queen[1]),Cord(one_queen[0],one_queen[1])
        
        #print(one_queen, old_queen)
        
        if(one_queen[1]>old_queen[1]):
            left=one_queen[1]-old_queen[1]
            #print("Extend to the left",left)

        if((len(span1[0])-one_queen[1]-1)>(len(span[0])-old_queen[1]-1)):
            right=(len(span1[0])-one_queen[1]-1)-(len(span[0])-old_queen[1]-1)
            #print("Extend to the right",right)
        if(one_queen[0]>old_queen[0]):
            top=one_queen[0]-old_queen[0]
            #print("Extend to the top",top)
        if((len(span1)-one_queen[0]-1)>(len(span)-old_queen[0]-1)):
            bottom=(len(span1)-one_queen[0]-1)-(len(span)-old_queen[0]-1)
            #print("Extend to the bottom",bottom) 
        #print(left,right,top,bottom)
       
        NewSpan=np.ones(((len(span)+top+bottom),(len(span[0])+right+left)))
        #print("Extending",len(span),"heightways by top:",top,"and bottom:",bottom,"widthways by right:",right,"and left:",left)
        
        #self.show_span(NewSpan)
        for a in range(len(span)):
            for b in range(len(span[a])):
                NewSpan[a+top][b+left]=span[a][b]
                #print(Cord(a+top,b+left).CordToString(),self.mapDict.get(Cord(a,b)))
                newDict[Cord(a+top,b+left)]=self.mapDict.get(Cord(a,b))
        els=list(newDict.keys())
        #print("a","newDict contains from",els[0].CordToString(),els[-1].CordToString())
                
        for key in self.historyInView:
            cell=self.historyInView.get(key)
            #self.historyInView[key]=Cord(cell.X+(old_queen[0]-one_queen[0])+top,cell.Y+(old_queen[1]-one_queen[1])+left) 
            self.historyInView[key]=Cord(cell.X+top,cell.Y+left)
            #print("Updating history in view:",cell.CordToString(),self.historyInView.get(key).CordToString())
        #print("b")
        #show_span(NewSpan)
        for i in range(len(span1)):
            #print("0")
            for j in range(len(span1[i])):                
                iNew=i+(old_queen[0]-one_queen[0])+top
                jNew=j+(old_queen[1]-one_queen[1])+left
                #print(i,j," which is",span1[i][j],"->",iNew,jNew,"which is",newDict.get(Cord(iNew,jNew)))
                #print("1")
                if Cord(iNew,jNew) in newDict:
                    #print("2")
                    #print(Cord(iNew,jNew).CordToString(),"is in mapDict",newDict.get(Cord(iNew,jNew)))
                    if(t1>newDict.get(Cord(iNew,jNew))[1]): 
                        if(span1[i][j]==7):
                            continue
                            #print("Updating",iNew,jNew,"to",span1[i][j],"at",t1,"from",newDict.get(Cord(iNew,jNew))[1])
                        NewSpan[iNew][jNew]=span1[i][j]
                        newDict[Cord(iNew,jNew)]=[span1[i][j],t1]
                else:
                    #print("3:",Cord(iNew,jNew).CordToString(),"not in mapDict",i,j,"->",iNew,jNew,":",(len(span)+top+bottom),(len(span[0])+right+left))
                    #print("3")
                    NewSpan[iNew][jNew]=span1[i][j]
                    newDict[Cord(iNew,jNew)]=[span1[i][j],t1]
        els=list(newDict.keys())
        #print("c","newDict contains from",els[0].CordToString(),els[-1].CordToString())           
        for s in range(len(NewSpan)):
            for t in range(len(NewSpan[i])):
                if Cord(s,t) not in newDict:#self.mapDict:
                    newDict[Cord(s,t)]=[1,0]
                    #print(Cord(s,t).CordToString(),"not in mapDict")
                    
                #print("("+str(i)+","+str(j)+")", span1[i][j], "("+str(iNew)+","+str(jNew)+")", NewSpan[iNew][jNew] )
        #print(" ")
        els=list(newDict.keys())
        #print("d, At the END","newDict contains from",els[0].CordToString(),els[-1].CordToString())
        #print("After merge")
        #self.show_span(NewSpan)
        self.mapDict=newDict
        self.view=NewSpan
        #return NewSpan, newDict
    
    def getView(self):
        return self.view
    
    def updateView(self,direction):
        if(self.view==[]):
            return
        #print("UPDATING")
        els=list(self.mapDict.keys())
        #print("a","mapDict contains from",els[0].CordToString(),els[-1].CordToString())
        for key in self.mapDict:
            cell=key
            #if(cell is None):
                #print("in update 2",cell.CordToString(),self.mapDict.get(cell))
            value=self.mapDict.get(cell)[0]
            #print("in update 3")
            timeStamp=self.mapDict.get(cell)[1]
            #print("in update 4")
            
            if(value==7):
                #print("Height:",len(self.view),"Width:",len(self.view[0]))
                #print("Setting",cell.CordToString(),"to 0")
                
                self.mapDict[cell]=[0,timeStamp]
                newCell=None
                if(direction==0):
                    newCell=Cord(cell.X,cell.Y+1)
                if(direction==1):
                    newCell=Cord(cell.X,cell.Y-1)
                if(direction==2):
                    newCell=Cord(cell.X+1,cell.Y)
                if(direction==3):
                    newCell=Cord(cell.X-1,cell.Y)
                if(direction==4):
                    newCell=Cord(cell.X,cell.Y)
                    
                
               # print("i",newCell.CordToString(),len(self.view),len(self.view[0]))
                
                edge,whichEdge=self.isQueenOnEdge()
                #print("Edge?",edge,"Which edge:",whichEdge,"Moving in dir:",direction)
                if(edge): #check ant doesn't walk out of edge of view
                    #print("WALKING OFF VIEW")
                    exSpan=[]
                    start=[0,0]
                    if direction in whichEdge:
                        if(direction==0):
                            #print("Extending from","(",len(self.view),",",len(self.view[0]),") to (",len(self.view),",",len(self.view[0])+1,")")
                            exSpan=np.ones((len(self.view),len(self.view[0])+1))
                            start=[0,0]
                            cell=Cord(cell.X,cell.Y)
                            newCell=Cord(newCell.X,newCell.Y)
                        if(direction==1):
                            #print("Extending from","(",len(self.view),",",len(self.view[0]),") to (",len(self.view),",",len(self.view[0])+1,")")
                            exSpan=np.ones((len(self.view),len(self.view[0])+1))
                            start=[0,1]
                            cell=Cord(cell.X,cell.Y+1)
                            newCell=Cord(newCell.X,newCell.Y+1)
                        if(direction==2):
                            #print("Extending from","(",len(self.view),",",len(self.view[0]),") to (",len(self.view)+1,",",len(self.view[0]),")")
                            exSpan=np.ones((len(self.view)+1,len(self.view[0])))
                            start=[0,0]
                            cell=Cord(cell.X,cell.Y)
                            newCell=Cord(newCell.X,newCell.Y)
                        if(direction==3):
                            #print("Extending from","(",len(self.view),",",len(self.view[0]),") to (",len(self.view)+1,",",len(self.view[0]),")")
                            exSpan=np.ones((len(self.view)+1,len(self.view[0])))
                            start=[1,0]
                            cell=Cord(cell.X+1,cell.Y)
                            newCell=Cord(newCell.X+1,newCell.Y)
                        if(direction==4):
                            #print("Extending from","(",len(self.view),",",len(self.view[0]),") to (",len(self.view),",",len(self.view[0]),")")
                            exSpan=np.ones((len(self.view),len(self.view[0])))
                            start=[0,0]
                   #     print("a",exSpan)
                        for a in range(len(self.view)):
                            for b in range(len(self.view[a])):
                                exSpan[a+start[0]][b+start[1]]=self.view[a][b]
                        #print("Combining")        
                        self.combine(exSpan,self.TimeStep-1)
                    #print("Finish walking off view")
                #self.show_span(self.view)
                #print("Setting",cell.CordToString(),"to 0", len(self.view))                
                self.view[cell.X][cell.Y]=0
                #self.show_span(self.view)
                #print("Setting",newCell.CordToString(),"to 7",len(self.view))
                self.view[newCell.X][newCell.Y]=7
                #self.show_span(self.view)
                #print("ii")
                self.mapDict[newCell]=[7,self.getTime()]
                #print("iii")
                self.historyInView[self.getTime()]=newCell
                #print("END UPDATING")
                break
     
    def isQueenOnEdge(self):
        #print("Testing if Queen on edge")
        view=self.view
        #self.show_span(view)
        #print(view)
        dir=[]
        edge=False
        dir.append(4)
        for cell in view[0]:
            if cell==7:
                edge=True
                dir.append(3)
        for cell in view[len(view)-1]:
            if cell==7:
                edge=True
                dir.append(2)
        for row in view:
            if row[0]==7:
                edge=True
                dir.append(1)
            if row[len(row)-1]==7:
                edge=True
                dir.append(0)
        if(edge):
            return True,dir
        else:
            return False,dir
  
    def getReward(self, pos, possible, oldPos, view, rNotPos, rWall, rEnt, rFinBef, rEx, rToEx, rRep, rElse):
        
        if not(possible):
            return rNotPos
        else:
            if(self.m.CheckWall(pos)):
                return rWall
            if(self.m.CheckEntrance(pos)):
                return rEnt
            if (oldPos.equals(pos) and self.m.CheckExit(pos)):
                    return rFinBef
            if(self.m.CheckExit(pos)):
                return rEx
            if(self.m.CheckClear(pos)):
                exits=self.m.Exits
                for row in view:
                    if (3 in row):
                        for exit in exits:
                            if((math.pow(exit.X-pos.X,2)+math.pow(exit.Y-pos.Y,2))<(math.pow(exit.X-oldPos.X,2)+math.pow(exit.Y-oldPos.Y,2))):
                                return rToEx
                if (oldPos.equals(pos)):
                    return rRep
                else:
                    return rElse
                
    def norm_view(self,m,n):
        arr=np.ones((m,n))
        span=self.view
        for a in range(len(span)):
            for b in range(len(span[a])):
                arr[a][b]=span[a][b]
        #print(arr)
        return arr

In [27]:
m= "Test,10,10,4,0,4,9,1111211111100000000111100110011000000001100001011111011000111000000001100101010110000000011111311111"
maze=Maze(m)
ss,t,pos=[[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [7, 0, 4, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 0, 0]] ,4, Cord(4,0)
q=Queen(maze)
q.combine(ss,t,pos)
q.norm_view(10,5)

[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[7, 0, 4, 0, 0]
[1, 0, 1, 0, 1]
[1, 0, 1, 0, 0]
 
[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[7, 0, 4, 0, 0]
[1, 0, 1, 0, 1]
[1, 0, 1, 0, 0]
 
0 0 0 0
[1. 0. 1. 0. 0.]
[1. 0. 0. 0. 0.]
[7. 0. 4. 0. 0.]
[1. 0. 1. 0. 1.]
[1. 0. 1. 0. 0.]
 
[[1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [7. 0. 4. 0. 0.]
 [1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
